<a href="https://colab.research.google.com/github/JalenCai/Sympify-AI/blob/main/Sympify_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
rm -r sample_data

In [ ]:
%%bash
cat > .gitignore
.*
.gitignore

In [ ]:
%%bash
git config --global user.email "caijalen1@gmail.com"
git config --global user.name "Jalen Cai"

In [ ]:
%%bash
git init

Initialized empty Git repository in /content/.git/


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>


In [ ]:
%%bash
git add -A

In [ ]:
%%bash
git commit -m "Fixed some of the code."

On branch main
nothing to commit, working tree clean


CalledProcessError: Command 'b'git commit -m "Fixed some of the code."\n'' returned non-zero exit status 1.

In [ ]:
%%bash
rm -r ~/.ssh/id_ed25519
ssh-keygen -t ed25519 -C '[caijalen1@gmail.com]' -f ~/.ssh/id_ed25519 -P ''
cat > ~/.ssh/config
Host *
  AddKeysToAgent yes
  IdentityFile ~/.ssh/id_ed25519

Generating public/private ed25519 key pair.
Your identification has been saved in /root/.ssh/id_ed25519
Your public key has been saved in /root/.ssh/id_ed25519.pub
The key fingerprint is:
SHA256:LkY1UYk6ije3M69fDmaoweyxMHO2tq2NYyBihne00Zg [caijalen1@gmail.com]
The key's randomart image is:
+--[ED25519 256]--+
|        .o..     |
|        ...      |
|     + .o        |
|    E +. .       |
|.  o +..S        |
|+o+o*..o         |
|+o++Bo+.= .      |
|   B+@++ +       |
|   oO++*o .      |
+----[SHA256]-----+


rm: cannot remove '/root/.ssh/id_ed25519': No such file or directory
Created directory '/root/.ssh'.


In [ ]:
%%bash
eval "$(ssh-agent -s)"
ssh-add ~/.ssh/id_ed25519

Agent pid 673


Identity added: /root/.ssh/id_ed25519 ([caijalen1@gmail.com])


In [ ]:
%%bash
cat ~/.ssh/id_ed25519.pub

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAII1nt5Y1wx5uMflbVaM9LO4YmfqlXQSaqpnyiKlH71/l [caijalen1@gmail.com]


In [ ]:
%%bash
ssh -o "StrictHostKeyChecking no" -T git@github.com | exit 0

Hi JalenCai! You've successfully authenticated, but GitHub does not provide shell access.


In [ ]:
%%bash
# Fetch the latest changes from the remote repository
git fetch origin

# Merge the changes into your local branch
git merge origin/main

# Push your changes to the remote repository
git push -u origin main


fatal: refusing to merge unrelated histories


CalledProcessError: Command 'b'# Fetch the latest changes from the remote repository\ngit fetch origin\n\n# Merge the changes into your local branch\ngit merge origin/main\n\n\n'' returned non-zero exit status 128.

In [ ]:
%%bash
git remote set-url origin git@github.com:JalenCai/Sympify-AI.git
git branch -M main
git push -u origin main

To github.com:JalenCai/Sympify-AI.git
 ! [rejected]        main -> main (fetch first)
error: failed to push some refs to 'github.com:JalenCai/Sympify-AI.git'
hint: Updates were rejected because the remote contains work that you do
hint: not have locally. This is usually caused by another repository pushing
hint: to the same ref. You may want to first integrate the remote changes
hint: (e.g., 'git pull ...') before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.


CalledProcessError: Command 'b'git remote set-url origin git@github.com:JalenCai/Sympify-AI.git\ngit branch -M main\ngit push -u origin main\n'' returned non-zero exit status 1.

In [1]:
import pandas as pd

# Load the database
file_path = '/content/Sympify_DB - Database.csv'
df = pd.read_csv(file_path)

# Combine symptom columns into a single list for each disease
symptom_columns = [f'Symptom {i}' for i in range(1, 20)]
df['Symptoms'] = df[symptom_columns].apply(lambda x: [symptom.lower() for symptom in x if pd.notnull(symptom)], axis=1)

# Function to get user symptoms
def get_user_symptoms():
    symptoms = input("What are your symptoms? (separate with commas): ").split(',')
    symptoms = [symptom.strip().lower() for symptom in symptoms]
    return symptoms

# Function to match symptoms
def match_symptoms(user_symptoms, df):
    matched_diseases = df[df['Symptoms'].apply(lambda x: any(symptom in x for symptom in user_symptoms))]
    return matched_diseases

# Function to rank diseases initially
def rank_diseases(matched_diseases, user_symptoms):
    matched_diseases['Score'] = matched_diseases['Symptoms'].apply(lambda x: sum(1 for symptom in user_symptoms if symptom in x))
    ranked_diseases = matched_diseases.sort_values(by='Score', ascending=False)
    return ranked_diseases

# Function to update probabilities in real-time based on user feedback
def update_probabilities(ranked_diseases, feedback, symptom):
    if feedback == 'yes':
        ranked_diseases['Score'] += ranked_diseases['Symptoms'].apply(lambda x: 1 if symptom in x else 0)
    elif feedback == 'no':
        ranked_diseases['Score'] -= ranked_diseases['Symptoms'].apply(lambda x: 1 if symptom in x else 0)
    elif feedback == 'maybe':
        ranked_diseases['Score'] += ranked_diseases['Symptoms'].apply(lambda x: 0.5 if symptom in x else 0)
    elif feedback == 'i don\'t know':
        ranked_diseases['Score'] += ranked_diseases['Symptoms'].apply(lambda x: 0 if symptom in x else 0)
    return ranked_diseases.sort_values(by='Score', ascending=False)

# Function to get user feedback one symptom at a time
def get_user_feedback(ranked_diseases):
    symptoms_asked = set()
    user_feedback = {}

    for index, row in ranked_diseases.iterrows():
        print(f"Disease: {row['Disease Name']}")
        for symptom in row['Symptoms']:
            if symptom in user_feedback:
                print(f"You have already responded to '{symptom}' as '{user_feedback[symptom]}'")
                continue
            if symptom not in symptoms_asked:
                feedback = input(f"Do you have the symptom '{symptom}'? (yes/no/maybe/i don't know/i do not want to continue): ")
                if feedback == 'i do not want to continue':
                    return None
                user_feedback[symptom] = feedback
                symptoms_asked.add(symptom)
                ranked_diseases = update_probabilities(ranked_diseases, feedback, symptom)

    return ranked_diseases

# Function to display results
def display_results(ranked_diseases):
    print("Final Results:")
    for index, row in ranked_diseases.iterrows():
        print(f"Disease: {row['Disease Name']} - Probability Score: {row['Score']}")

# Main function
def main():
    user_symptoms = get_user_symptoms()
    matched_diseases = match_symptoms(user_symptoms, df)
    ranked_diseases = rank_diseases(matched_diseases, user_symptoms)
    while True:
        ranked_diseases = get_user_feedback(ranked_diseases)
        if ranked_diseases is None:
            print("You have chosen to stop the program.")
            break
        display_results(ranked_diseases)

# Run the main function
main()


KeyError: "['Symptom 1'] not in index"

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 14.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
import spacy
import numpy as np


!pip install spacy
!python -m spacy download en_core_web_md

# Load the spaCy model
nlp = spacy.load("en_core_web_md")

# Load the database
file_path = '/content/Sympify_DB - Sheet1 (1).csv'
df = pd.read_csv(file_path)

# Combine symptom columns into a single list for each disease
symptom_columns = [f'Symptom {i}' for i in range(1, 20)]
df['Symptoms'] = df[symptom_columns].apply(lambda x: [symptom.lower() for symptom in x if pd.notnull(symptom)], axis=1)

# List of all symptoms from the dataset
all_symptoms = list(set(symptom for sublist in df['Symptoms'] for symptom in sublist))

# Precompute vectors for all symptoms
symptom_vectors = {symptom: nlp(symptom).vector for symptom in all_symptoms}

# Function to find the most similar term in a list
def most_similar(term, symptom_vectors):
    term_vec = nlp(term).vector
    similarities = {other: np.dot(term_vec, vec) / (np.linalg.norm(term_vec) * np.linalg.norm(vec)) for other, vec in symptom_vectors.items()}
    most_similar_term = max(similarities, key=similarities.get)
    return most_similar_term

# Function to normalize symptoms using word embeddings
def normalize_symptom_embedding(symptom, symptom_vectors):
    return most_similar(symptom, symptom_vectors)

# Function to preprocess symptoms
def preprocess_symptoms(symptoms):
    return [normalize_symptom_embedding(symptom.lower().strip(), symptom_vectors) for symptom in symptoms]

# Function to get user symptoms
def get_user_symptoms():
    symptoms = input("What are your symptoms? (separate with commas): ").split(',')
    symptoms = preprocess_symptoms(symptoms)
    return symptoms

# Update the dataframe with normalized symptoms
df['Normalized Symptoms'] = df['Symptoms'].apply(lambda x: preprocess_symptoms(x))

# Function to match symptoms
def match_symptoms(user_symptoms, df):
    matched_diseases = df[df['Normalized Symptoms'].apply(lambda x: any(symptom in x for symptom in user_symptoms))]
    return matched_diseases

# Function to rank diseases initially
def rank_diseases(matched_diseases, user_symptoms):
    matched_diseases['Score'] = matched_diseases['Normalized Symptoms'].apply(lambda x: sum(1 for symptom in user_symptoms if symptom in x))
    ranked_diseases = matched_diseases.sort_values(by='Score', ascending=False)
    return ranked_diseases

# Function to update probabilities in real-time based on user feedback
def update_probabilities(ranked_diseases, feedback, symptom):
    normalized_symptom = normalize_symptom_embedding(symptom, symptom_vectors)
    if feedback == 'yes':
        ranked_diseases['Score'] += ranked_diseases['Normalized Symptoms'].apply(lambda x: 1 if normalized_symptom in x else 0)
    elif feedback == 'no':
        ranked_diseases['Score'] -= ranked_diseases['Normalized Symptoms'].apply(lambda x: 1 if normalized_symptom in x else 0)
    elif feedback == 'maybe' or feedback == 'i don\'t know':
        ranked_diseases['Score'] += ranked_diseases['Normalized Symptoms'].apply(lambda x: 0.5 if normalized_symptom in x else 0)
    return ranked_diseases.sort_values(by='Score', ascending=False)

# Function to get user feedback one symptom at a time
def get_user_feedback(ranked_diseases):
    symptoms_asked = set()
    user_feedback = {}

    # Iterate over diseases
    for index, row in ranked_diseases.iterrows():
        print(f"Disease: {row['Disease Name']}")
        # Iterate over symptoms of each disease
        for symptom in row['Normalized Symptoms']:
            if symptom in user_feedback:
                print(f"You have already responded to '{symptom}' as '{user_feedback[symptom]}'")
                continue
            if symptom not in symptoms_asked:
                feedback = input(f"Do you have the symptom '{symptom}'? (yes/no/maybe/i don't know/i do not want to continue): ").strip().lower()
                if feedback == 'i do not want to continue':
                    return None
                user_feedback[symptom] = feedback
                symptoms_asked.add(symptom)
                ranked_diseases = update_probabilities(ranked_diseases, feedback, symptom)

    return ranked_diseases

# Function to display results
def display_results(ranked_diseases):
    print("Final Results:")
    for index, row in ranked_diseases.iterrows():
        print(f"Disease: {row['Disease Name']} - Probability Score: {row['Score']}")

# Main function
def main():
    user_symptoms = get_user_symptoms()
    matched_diseases = match_symptoms(user_symptoms, df)
    ranked_diseases = rank_diseases(matched_diseases, user_symptoms)
    while True:
        ranked_diseases = get_user_feedback(ranked_diseases)
        if ranked_diseases is None:
            print("You have chosen to stop the program.")
            break
        display_results(ranked_diseases)

# Run the main function
main()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 16.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import spacy
import numpy as np

# Load the spaCy model
nlp = spacy.load("en_core_web_md")

# Load the database
file_path = '/content/Sympify_DB - Sheet1 (1).csv'
df = pd.read_csv(file_path)

# Combine symptom columns into a single list for each disease
symptom_columns = [f'Symptom {i}' for i in range(1, 20)]
df['Symptoms'] = df[symptom_columns].apply(lambda x: [symptom.lower() for symptom in x if pd.notnull(symptom)], axis=1)

# List of all symptoms from the dataset
all_symptoms = list(set(symptom for sublist in df['Symptoms'] for symptom in sublist))

# Precompute vectors for all symptoms
symptom_vectors = {symptom: nlp(symptom).vector for symptom in all_symptoms}

# Function to find the most similar term in a list
def most_similar(term, symptom_vectors):
    term_vec = nlp(term.lower()).vector
    similarities = {other: np.dot(term_vec, vec) / (np.linalg.norm(term_vec) * np.linalg.norm(vec)) for other, vec in symptom_vectors.items()}
    most_similar_term = max(similarities, key=similarities.get)
    return most_similar_term

# Function to normalize symptoms using word embeddings
def normalize_symptom_embedding(symptom, symptom_vectors):
    return most_similar(symptom, symptom_vectors)

# Function to preprocess symptoms
def preprocess_symptoms(symptoms):
    return [normalize_symptom_embedding(symptom.lower().strip(), symptom_vectors) for symptom in symptoms]

# Function to get user symptoms
def get_user_symptoms():
    symptoms = input("What are your symptoms? (separate with commas): ").split(',')
    symptoms = preprocess_symptoms(symptoms)
    return symptoms

# Update the dataframe with normalized symptoms
df['Normalized Symptoms'] = df['Symptoms'].apply(lambda x: preprocess_symptoms(x))

# Function to match symptoms
def match_symptoms(user_symptoms, df):
    matched_diseases = df[df['Normalized Symptoms'].apply(lambda x: any(symptom in x for symptom in user_symptoms))]
    return matched_diseases

# Function to rank diseases initially
def rank_diseases(matched_diseases, user_symptoms):
    matched_diseases['Initial Score'] = matched_diseases['Normalized Symptoms'].apply(lambda x: sum(1 for symptom in user_symptoms if symptom in x))
    return matched_diseases

# Function to update probabilities in real-time based on user feedback
def update_probabilities(ranked_diseases, feedback, symptom):
    normalized_symptom = normalize_symptom_embedding(symptom, symptom_vectors)
    if feedback == 'yes':
        ranked_diseases['Adjusted Score'] += ranked_diseases['Normalized Symptoms'].apply(lambda x: 1 if normalized_symptom in x else 0)
    elif feedback == 'no':
        ranked_diseases['Adjusted Score'] -= ranked_diseases['Normalized Symptoms'].apply(lambda x: 1 if normalized_symptom in x else 0)
    elif feedback == 'maybe' or feedback == 'i don\'t know':
        ranked_diseases['Adjusted Score'] += ranked_diseases['Normalized Symptoms'].apply(lambda x: 0.5 if normalized_symptom in x else 0)
    # Ensure scores remain non-negative
    ranked_diseases['Adjusted Score'] = ranked_diseases['Adjusted Score'].apply(lambda x: max(0, x))
    return ranked_diseases

# Function to normalize scores to percentages
def normalize_scores_to_percentage(ranked_diseases):
    total_score = ranked_diseases['Adjusted Score'].sum()
    if total_score > 0:
        ranked_diseases['Percentage'] = (ranked_diseases['Adjusted Score'] / total_score) * 100
    else:
        ranked_diseases['Percentage'] = 0
    return ranked_diseases

# Function to get user feedback one symptom at a time
def get_user_feedback(ranked_diseases):
    symptoms_asked = set()
    user_feedback = {}
    ranked_diseases['Adjusted Score'] = ranked_diseases['Initial Score']

    # Iterate over diseases
    for index, row in ranked_diseases.iterrows():
        print(f"Disease: {row['Disease Name']}")
        # Iterate over symptoms of each disease
        for symptom in row['Normalized Symptoms']:
            if symptom in user_feedback:
                print(f"You have already responded to '{symptom}' as '{user_feedback[symptom]}'")
                continue
            if symptom not in symptoms_asked:
                feedback = input(f"Do you have the symptom '{symptom}'? (yes/no/maybe/i don't know/i do not want to continue): ").strip().lower()
                if feedback == 'i do not want to continue':
                    return None
                user_feedback[symptom] = feedback
                symptoms_asked.add(symptom)
                ranked_diseases = update_probabilities(ranked_diseases, feedback, symptom)

    # Normalize scores to percentages
    ranked_diseases = normalize_scores_to_percentage(ranked_diseases)
    return ranked_diseases

# Function to display results
def display_results(ranked_diseases):
    print("Final Results:")
    for index, row in ranked_diseases.iterrows():
        print(f"Disease: {row['Disease Name']} - Probability Score: {row['Percentage']:.2f}%")

# Main function
def main():
    user_symptoms = get_user_symptoms()
    matched_diseases = match_symptoms(user_symptoms, df)
    ranked_diseases = rank_diseases(matched_diseases, user_symptoms)
    while True:
        ranked_diseases = get_user_feedback(ranked_diseases)
        if ranked_diseases is None:
            print("You have chosen to stop the program.")
            break
        display_results(ranked_diseases)

# Run the main function
main()


KeyboardInterrupt: Interrupted by user

In [ ]:
%%bash


bash: line 1: syntax error near unexpected token `'Hello''
bash: line 1: `print('Hello')'


CalledProcessError: Command 'b"print('Hello')\n"' returned non-zero exit status 2.